In [1]:
import os
import numpy as np
import imageio
from matplotlib import pyplot as plt
from utils import *

In [2]:
tMax = 60
stock = 340
strikeMin = stock*0.93
strikeMax = stock*1.07

###
n = int((strikeMax - strikeMin)/0.01 + 1)
strike = np.linspace(strikeMin, strikeMax, n)

### t days until expiration
valueT = np.zeros((tMax+2, n))
valueT[tMax+1] = BlackScholes(strike, stock, tMax+1)
thetaT = np.zeros((tMax+1, n))
thetaFactorT = np.zeros((tMax+1, n))
for t in range(tMax+1)[::-1]:
    value = BlackScholes(strike, stock, t)
    valueT[t] = value
    prev = valueT[t+1]
    theta = value - prev
    thetaT[t] = theta
    thetaFactorT[t] = theta/prev

### Normalizing Data

# normalizing stock value
moneyness = (stock - strike) / stock

# normalizing contract value
idx = np.argmin(np.abs(stock - strike))
yArr = valueT / valueT[tMax, idx]
thetaT = thetaT / valueT[tMax, idx]
thetaMax = np.max(-thetaT)
thetaMax = 0.125 #makes plot look prettier

In [3]:
# Generating images
files = []
for t in range(tMax+1)[::-1]:
    fig, ax1 = plt.subplots(figsize = [10, 8], tight_layout = True)
    xlabel = r"$\bf{Option\ Moneyness}$"
    xlabel += "\n"
    xlabel += r"$\rm{\left(\frac{stock\ price\ \minus\ strike\ price}{stock\ price}\right)}$"
    plt.xlabel(xlabel, fontsize = 16)    

    title = r"$\bf{Daily\ Theta\ Decay\ of\ Call\ Option\ Contracts}$"
    title += "\n"
    title += r"$\rm{Days\ to\ Expiration:}$"
    title += " %2s" % t
    plt.title(title, family = "monospace", fontsize = 16)
    
    xMin, xMax = np.min(moneyness), np.max(moneyness)
    plt.xlim(xMin, xMax)
    ticks = plt.gca().get_xticks()
    ticks = ticks[(ticks <= xMax)&(ticks >= xMin)]
    xLabels = []
    for curLabel in ticks:
        newLabel = "{:.0f}%\n".format(curLabel*100)
        if curLabel == 0:
            newLabel += r"$\it{ATM}$"
        elif curLabel == ticks[0]:
            newLabel += r"$\it{OTM}$"
        elif curLabel == ticks[-1]:
            newLabel += r"$\it{ITM}$"
        xLabels.append(newLabel)
    plt.xticks(ticks, xLabels, fontsize = 14)
    plt.grid()
    
    ax1.tick_params(axis='y', labelsize = 14, labelcolor='r')
    ax1.set_yticks(np.linspace(-thetaMax, 0, 6))
    ax1.plot(moneyness, thetaT[t], color = 'r', linewidth = 2.5)
    ylabel = r"$\bf{Theta}$"
    ylabel += "\n"
    ylabel += r"$\rm{(Daily\ decline\ in\ option\ value)}$"
    ax1.set_ylabel(ylabel, color = 'r', fontsize = 16)
    ax1.set_ylim(-thetaMax, 0)
    
    ax2 = ax1.twinx()
    ax2.plot(moneyness, thetaFactorT[t], color = 'b', linewidth = 2.5)
    ax2.tick_params(axis='y', labelsize = 14, labelcolor='b')
    ax2.set_yticks(np.linspace(-1, 0, 6))
    ylabel = r"$\bf{Percentage\ Theta\ Loss}$"
    ylabel += "\n"
    ylabel += r"$\rm{\left(\frac{Theta}{option\ price}\right)}$"
    ax2.set_ylabel(ylabel, color = 'b', fontsize = 16)
    ax2.set_ylim(-1, 0)
    plt.gca().set_yticklabels(['{:.0f}%'.format(y*100) for y in plt.gca().get_yticks()]);
    
    fname = "theta_%02d.png" % t
    files.append(fname)
    plt.savefig(fname)
    plt.close()

In [4]:
### Making movie animation
images = []
for i in range(tMax+1):
    fname = files[i]
    # make first and last frames longer
    if i == 0 or i == tMax:
        for _ in range(9):
            images.append(imageio.imread(fname))
    #
    images.append(imageio.imread(fname))
imageio.mimsave('Theta.gif', images, duration=0.1)

### cleanup
for fname in files:
    os.remove(fname)